In [ ]:
!pip install geopandas

In [113]:
#Importing libraries
import pandas as pd
import geopandas as gpd
import numpy as np

In [114]:
#Importing crime data and opening the weather data file:
#!wget https://data.buffalony.gov/api/views/d6g9-xbgu/rows.csv?accessType=DOWNLOAD;
#Reading the crime data into a pandas dataframe :
crime_df = pd.read_csv('/content/rows.csv?accessType=DOWNLOAD')
#converting the incident_datetime column into datetime format:
crime_df['incident_datetime']=pd.to_datetime(crime_df['incident_datetime'])
# Sorting the data by time :
crime_df.sort_values('incident_datetime',inplace=True)
# Creating a new neighborhood column with empty values :
crime_df['neighborhood']=np.nan

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [117]:
#Reading the geojson data provided by https://data.buffalony.gov "https://data.buffalony.gov/Economic-Neighborhood-Development/Neighborhoods/q9bk-zu3p" :
polygon = gpd.read_file("/content/Neighborhoods.geojson")
# Converting the pandas dataframe into a geopandas dataframe(has higher support for geospatial data) :
gdf = gpd.GeoDataFrame( crime_df, geometry=gpd.points_from_xy(crime_df.longitude, crime_df.latitude))

In [118]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'neighborhood'] = polygon['nbhdname'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
gdf.drop('geometry',axis=1,inplace=True)

In [131]:
#I removed the last 5 rows as they didnt have coordinates either:
gdf[-50:-5]

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,zip,country,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood
12852,NaN,21-0390633,2021-02-08 18:03:36,ASSAULT,Buffalo Police are investigating this report o...,NaN,200 Block PERSON ST,NaN,Buffalo,NY,NaN,NaN,42.897000,-78.825000,02/08/2021 06:03:36 PM,NaN,POINT (-78.825 42.897),18,MONDAY,Assault,28,2010,2,Genesee-Moselle,District C,FILLMORE,Genesee-Moselle
12853,NaN,21-0390652,2021-02-08 18:21:32,ROBBERY,Buffalo Police are investigating this report o...,NaN,WALDEN AV & KIEFER ST,NaN,Buffalo,NY,NaN,NaN,42.904000,-78.832000,02/08/2021 06:22:32 PM,NaN,POINT (-78.832 42.904),18,MONDAY,Robbery,27.02,1003,1,Broadway Fillmore,District C,FILLMORE,MLK Park
12854,NaN,21-0390670,2021-02-08 18:37:18,UUV,Buffalo Police are investigating this report o...,NaN,400 Block WINSLOW AV,NaN,Buffalo,NY,NaN,NaN,42.913000,-78.835000,02/08/2021 06:38:18 PM,NaN,POINT (-78.835 42.913),18,MONDAY,Theft of Vehicle,34,4002,4,MLK Park,District C,MASTEN,MLK Park
12855,NaN,21-0390686,2021-02-08 18:50:00,UUV,Buffalo Police are investigating this report o...,NaN,500 Block NIAGARA ST,NaN,Buffalo,NY,NaN,NaN,NaN,NaN,02/08/2021 08:00:00 PM,NaN,NaN,20,MONDAY,Theft of Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12856,NaN,21-0390705,2021-02-08 19:17:00,UUV,Buffalo Police are investigating this report o...,NaN,PETER ST & AMHERST ST,NaN,Buffalo,NY,NaN,NaN,42.939000,-78.888000,02/08/2021 07:18:00 PM,NaN,POINT (-78.888 42.939),19,MONDAY,Theft of Vehicle,55,2001,2,Grant-Amherst,District D,NORTH,Grant-Amherst
12858,NaN,21-0390736,2021-02-08 19:52:54,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,300 Block ELMWOOD AV,NaN,Buffalo,NY,NaN,NaN,42.906000,-78.877000,02/08/2021 07:53:54 PM,NaN,POINT (-78.877 42.906),19,MONDAY,Theft,67.02,3002,3,Elmwood Bryant,District B,NIAGARA,Elmwood Bryant
12859,NaN,21-0390745,2021-02-08 20:15:00,ROBBERY,Buffalo Police are investigating this report o...,NaN,HIGHGATE AV & BAILEY AV,NaN,Buffalo,NY,NaN,NaN,42.948000,-78.814000,02/08/2021 08:15:47 PM,NaN,POINT (-78.814 42.948),20,MONDAY,Robbery,47,1000,1,University Heights,District E,UNIVERSITY,University Heights
12860,NaN,21-0390760,2021-02-08 20:38:33,UUV,Buffalo Police are investigating this report o...,NaN,700 Block MCKINLEY PW,NaN,Buffalo,NY,NaN,NaN,42.852000,-78.821000,02/08/2021 08:39:33 PM,NaN,POINT (-78.821 42.852),20,MONDAY,Theft of Vehicle,8,2007,2,South Park,District A,SOUTH,South Park
12861,NaN,21-0390776,2021-02-08 20:59:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,1500 Block HERTEL AV,NaN,Buffalo,NY,NaN,NaN,42.948000,-78.851000,02/08/2021 09:06:00 PM,NaN,POINT (-78.851 42.948),21,MONDAY,Theft,48,1003,1,North Park,District D,DELAWARE,North Park
12862,NaN,21-0390860,2021-02-08 23:07:00,ASSAULT,Buffalo Police are investigating this report o...,NaN,100 Block W FERRY ST,NaN,Buffalo,NY,NaN,NaN,42.915000,-78.896000,02/08/2021 11:15:00 PM,NaN,POINT (-78.896 42.915),23,MONDAY,Assault,61,1027,1,Upper West Side,District D,NIAGARA,Upper West Side


In [127]:
# checking null values:
gdf.isna().sum()

incident_id               12627
case_number                   0
incident_datetime             5
incident_type_primary         0
incident_description          0
clearance_type           267753
address_1                    39
address_2                267753
city                          0
state                         0
zip                      241403
country                  233464
latitude                    555
longitude                   555
created_at                    0
updated_at                12627
location                   2021
hour_of_day                   0
day_of_week                   5
parent_incident_type          0
Census Tract 1           255267
Census Block 1           255267
Census Block Group 1     255267
Neighborhood 1           255267
Police District 1        255267
Council District 1       255292
neighborhood               3410
dtype: int64

In [130]:
import nbconvert
!jupyter nbconvert --to html crime_geospatial.ipynb

[NbConvertApp] Converting notebook crime_geospatial.ipynb to html
[NbConvertApp] Writing 291962 bytes to crime_geospatial.html
